In [1]:
#libraries
import numpy as np
import random
import pandas as pd
import json

In [2]:
data = pd.read_csv('content_data.csv', delimiter = ';')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data.Genre.unique()

array(['Indie', 'Pop', 'Hip Hop/ Rap', 'Rock', 'Rock\xa0', 'Techno'],
      dtype=object)

In [3]:
#rename genre for songs with whitespace behind 'Rock' (see unique genres)
data.loc[list(data[data.Genre == 'Rock '].index),'Genre'] = 'Rock'
#remove rows with nan values (see unique genre values)
data = data[~data.Genre.isna()]
data = data.reset_index()

In [4]:
#Defining metrics for personas (1,2,3)
song_range = {'1':[1000,3000],
              '2':[1000,3000],
              '3':[200,500]}
weights_gen = {'1':[40,5,5,40,10],
               '2':[0,40,40,10,10],
               '3':[30,30,5,5,30]}
weights_known = {'1':[20,80],
                 '2':[70,30],
                 '3':[80,20]}

In [5]:
#pass which persona you want to have returned (input: '1','2','3')
def Persona(persona):
    genres = ['Indie', 'Pop', 'Hip Hop/ Rap', 'Rock', 'Techno']
    popularity = ['wellknown', 'lesser-known']
    
    #Retrieve number of songs and weights for respective persona
    n_songs = random.randint(song_range[persona][0],
                             song_range[persona][1])
    my_weights_gen = weights_gen[persona]
    my_weights_known = weights_known[persona]
    
    #create empty dataframe to fill later
    songs = pd.DataFrame()
    
    #each iteration simulates one song listened to
    for n in range(n_songs):
        #pick genre and if song is popular or not based on Persona odds
        genre = random.choices(genres,
                               weights = my_weights_gen,
                               k = 1)
        popular = random.choices(popularity,
                                weights = my_weights_known,
                                k=1)
        #create sub df of possible songs for given genre and popularity
        if popular[0] == 'wellknown':
            df_sub = data[(data['Genre'] == genre[0]) & (data['Popularity'] > 30)]
        else:
            df_sub = data[(data['Genre'] == genre[0]) & (data['Popularity'] < 30)]
        
        #pick random index from indexes matchin the criteria (of sub_df)
        index = random.sample(list(df_sub.index),
                              k = 1)
        
        #pick song with that index from data
        song = data[data.index == index[0]]
        #add song to songs
        songs = pd.concat([songs,song])
    
    #create count of each element (to eliminate duplicates but get count of each song)
    persona = songs.groupby(songs.columns.tolist(),as_index=False).size()
    #rename 'size' column
    persona = persona.rename(columns = {'size':'Listening frequency'})
    #sort df
    persona = persona.sort_values(by = ['Listening frequency'], ascending = False)
    return persona

Note: The generation of personas takes some time, since we are simulating 100 users per persona. We will add the file 'user_data.csv' to our hand in. Feel free to change the 'user_per_persona' number when you want to test the code.

In [6]:
#create empty user df to fill below
user_data = pd.DataFrame()
#define personas to iterate through
personas = ['1','2','3']

#define number of simulations per persona
n_user_per_persona = 100

#dummy to make user ids unique across personas
user_id = 1

for persona in personas:
    for i in range(n_user_per_persona):
        user = Persona(persona)
        user['User-ID'] = i + user_id
        user_data = pd.concat([user_data, user])
    user_id = user_id + n_user_per_persona

#normalize listening frequency
user_data['%_Frequency'] = user_data.groupby(['User-ID'], group_keys=False)['Listening frequency'].apply(lambda x: x*100 / sum(x))

In [7]:
user_data

,index,Artist,Top Track,Popularity,Genre,Image-URL-L,Listening frequency,User-ID,%_Frequency
201,300,Ganser,People Watching,17,Rock,https://i.scdn.co/image/ab67616d0000b273fd55ac...,25,1,1.185396
205,304,Ganser,Lucky,17,Rock,https://i.scdn.co/image/ab67616d0000b273d4c89f...,25,1,1.185396
222,321,Gauche,Flash,8,Rock,https://i.scdn.co/image/ab67616d00001e027013d6...,23,1,1.090564
6,6,Songs Of Water,Everything That Rises,24,Indie,"https://cdn.musicbed.com/image/upload/c_fill,d...",21,1,0.995733
18,18,Moonface,Minotaur Forgiving Minos,21,Indie,https://f4.bcbits.com/img/a3775302856_10.jpg,21,1,0.995733
...,...,...,...,...,...,...,...,...,...
108,223,Blake Shelton,Ol' Red,73,Hip Hop/ Rap,https://i.scdn.co/image/ab67616d0000b2731716c3...,1,300,0.302115
107,212,Teknical Development.is,Move with Us,27,Hip Hop/ Rap,https://i.scdn.co/image/ab67616d0000b27368eb79...,1,300,0.302115
106,199,Dua Lipa,Levitating,89,Pop,https://i.scdn.co/image/ab67616d0000b273bd26ed...,1,300,0.302115
31,66,Mac DeMarco,Chamber Of Reflection,83,Indie,https://ukutabs.com/wp-content/uploads/2021/08...,1,300,0.302115


In [8]:
user_data.to_csv('user_data.csv')